In [1]:
!pip install cdsapi

  Using cached cdsapi-0.5.1-py2.py3-none-any.whl


In [2]:
# General libs for file paths, data extraction, etc
from glob import glob
from pathlib import Path
from os.path import basename
import zipfile # To extract zipfiles
import urllib3 
urllib3.disable_warnings() # Disable warnings for data download via API

# CDS API
import cdsapi

# Libraries for working with multi-dimensional arrays
import numpy as np
import xarray as xr
import pandas as pd

# Libraries for plotting and visualising data
import matplotlib.path as mpath
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.feature as cfeature

In [3]:
URL = 'https://cds.climate.copernicus.eu/api/v2'
KEY = '140375:f24956b6-43ca-40ed-9563-be7031a4b2c3'

In [5]:
DATADIR = '/Users/fionaspuler/Desktop/ECMWF/code_and_data/testing_data_august'

In [15]:
experiments = ['historical', 'ssp5_8_5']
models = ['ipsl_cm6a_lr']
variables = ['near_surface_air_temperature','precipitation',
            'near_surface_wind_speed', 'daily_minimum_near_surface_air_temperature']
# area: [
#                53, 8.5, 50.5,
#                14.5,
#            ]

In [ ]:
c = cdsapi.Client(url=URL, key=KEY)


for v in variables:
    c.retrieve(
        'projections-cmip6',
        {
            'temporal_resolution': 'daily',
            'experiment': 'ssp5_8_5',
            'level': 'single_levels',
            'variable': v,
            'model': 'ipsl_cm6a_lr',
            'date': '2050-01-01/2070-12-31',
            'area': [
                    52, 8, 48,
                    12,
            ],
            'format': 'zip',
        },
        '/Users/fionaspuler/Desktop/ECMWF/code_and_data/testing_data_august/cmip6_daily_2050-2070_ipsl_'+experiment+'_'+v+'.zip')

In [ ]:
#https://confluence.ecmwf.int/pages/viewpage.action?pageId=228867588

import cdsapi
import requests
import iris
 
# CDS API script to use CDS service to retrieve daily ERA5* variables and iterate over
# all months in the specified years.
 
# Requires:
# 1) the CDS API to be installed and working on your system
# 2) You have agreed to the ERA5 Licence (via the CDS web page)
# 3) Selection of required variable, daily statistic, etc
 
# Output:
# 1) separate netCDF file for chosen daily statistic/variable for each month
 
c = cdsapi.Client(url=URL, key=KEY, timeout=300)
 
# Uncomment years as required
 
years =  ['1979', '1980', '1981',
            '1982', '1983', '1984',
            '1985', '1986', '1987',
            '1988', '1989', '1990',
            '1991', '1992', '1993',
            '1994', '1995', '1996',
            '1997', '1998', '1999',
            '2000', '2001', '2002',
            '2003', '2004', '2005',
            '2006', '2007', '2008',
            '2009', '2010', '2011',
            '2012', '2013', '2014', '2015']

months = ['01', '02', '03',
            '04', '05', '06',
            '07', '08', '09',
            '10', '11', '12']
 
# For valid keywords, see Table 2 of:
# https://datastore.copernicus-climate.eu/documents/app-c3s-daily-era5-statistics/C3S_Application-Documentation_ERA5-daily-statistics-v2.pdf
 
# select your variable; name must be a valid ERA5 CDS API name.
var = "2m_temperature"
 
# Select the required statistic, valid names given in link above
stat = "daily_mean"
 
# Loop over years and months

locations = []

for yr in years:
    for mn in months:
        result = c.service(
        "tool.toolbox.orchestrator.workflow",
        params={
             "realm": "user-apps",
             "project": "app-c3s-daily-era5-statistics",
             "version": "master",
             "kwargs": {
                 "dataset": "reanalysis-era5-single-levels",
                 "product_type": "reanalysis",
                 "variable": var,
                 "statistic": stat,
                 "year": yr,
                 "month": mn,
                 "time_zone": "UTC+00:0",
                 "frequency": "1-hourly",
                 "grid": "1.0/1.0",
                 "area":{"lat": [48, 52], "lon": [8, 12]}
                 },
        "workflow_name": "application"
        })
         
# set name of output file for each month (statistic, variable, year, month)     
   
 
        file_name = "/Users/fionaspuler/Desktop/ECMWF/code_and_data/testing_data_august/era5_" + stat + "_" + var + "_" + yr + "_" + mn + ".nc"
         
        location=result[0]['location']
        res = requests.get(location, stream = True)
        print("Writing data to " + file_name)
        with open(file_name,'wb') as fh:
            for r in res.iter_content(chunk_size = 1024):
                fh.write(r)
        fh.close()

2022-08-18 21:08:55,357 INFO Welcome to the CDS
2022-08-18 21:08:55,361 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/tasks/services/tool/toolbox/orchestrator/workflow/clientid-52eaeeb338c1425e8e33a85ec90fc4b7
2022-08-18 21:08:55,458 INFO Request is queued
2022-08-18 21:08:56,536 INFO Request is running
2022-08-18 21:09:03,880 INFO Request is completed
2022-08-18 21:09:04,238 INFO Welcome to the CDS
2022-08-18 21:09:04,240 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/tasks/services/tool/toolbox/orchestrator/workflow/clientid-f046a3a2f19947479d8f225dee531f8f
2022-08-18 21:09:04,323 INFO Request is queued


Writing data to /Users/fionaspuler/Desktop/ECMWF/code_and_data/testing_data_august/era5_daily_mean_2m_temperature_1979_01.nc


2022-08-18 21:09:05,385 INFO Request is running
2022-08-18 21:09:12,727 INFO Request is completed
2022-08-18 21:09:13,065 INFO Welcome to the CDS
2022-08-18 21:09:13,066 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/tasks/services/tool/toolbox/orchestrator/workflow/clientid-707c71ca4f67477499be3103e5ea55e3
2022-08-18 21:09:13,147 INFO Request is queued


Writing data to /Users/fionaspuler/Desktop/ECMWF/code_and_data/testing_data_august/era5_daily_mean_2m_temperature_1979_02.nc


2022-08-18 21:09:14,210 INFO Request is running
2022-08-18 21:09:26,678 INFO Request is completed
2022-08-18 21:09:27,082 INFO Welcome to the CDS
2022-08-18 21:09:27,083 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/tasks/services/tool/toolbox/orchestrator/workflow/clientid-5cf199632ecd4e1c9b14c42304324e92


...

2022-08-18 23:28:35,214 INFO Request is running
2022-08-18 23:29:49,772 INFO Request is completed
2022-08-18 23:29:50,121 INFO Welcome to the CDS
2022-08-18 23:29:50,124 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/tasks/services/tool/toolbox/orchestrator/workflow/clientid-da97f48830b2407296c7a51cc509f927
2022-08-18 23:29:50,208 INFO Request is queued


Writing data to /Users/fionaspuler/Desktop/ECMWF/code_and_data/testing_data_august/era5_daily_mean_2m_temperature_1989_10.nc


2022-08-18 23:29:51,290 INFO Request is running
2022-08-18 23:31:05,764 INFO Request is completed
2022-08-18 23:31:06,094 INFO Welcome to the CDS
2022-08-18 23:31:06,095 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/tasks/services/tool/toolbox/orchestrator/workflow/clientid-819d44c81de04329b7f3884eb06cb9d9
2022-08-18 23:31:06,164 INFO Request is queued


Writing data to /Users/fionaspuler/Desktop/ECMWF/code_and_data/testing_data_august/era5_daily_mean_2m_temperature_1989_11.nc


2022-08-18 23:31:07,234 INFO Request is running
